## WideNet prototype

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30505 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 6)
[+] Number of entries in each category:
[+] Training:
 [ 7734.  7422.  4828. 10849.  1077.    90.]
[+] Validation:
 [1910. 1848. 1201. 2775.  246.   20.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


In [3]:
# second embedding matrix for non-static channel
embeddings_index = {}
f = open(GLOVE_DIR)
print("[i] Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print("Done.\n[+] Proceeding with Embedding Matrix...", end="")
embedding_matrix_ns = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_ns[i] = embedding_vector
print("Completed!")

[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!


In [37]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# static channel
embedding_layer_frozen = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
embedded_sequences_frozen = embedding_layer_frozen(sequence_input)

# non-static channel
embedding_layer_train = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_ns],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences_train = embedding_layer_train(sequence_input)

First Half: LSTM > CNN

In [38]:
l_lstm1f = Bidirectional(LSTM(6,return_sequences=True))(embedded_sequences_frozen)
l_lstm1t = Bidirectional(LSTM(6,return_sequences=True))(embedded_sequences_train)
l_lstm1 = Concatenate(axis=1)([l_lstm1f, l_lstm1t])

In [39]:
l_conv_7 = Conv1D(filters=24,kernel_size=7,activation='relu')(l_lstm1)
l_conv_6 = Conv1D(filters=24,kernel_size=6,activation='relu')(l_lstm1)

l_conv_8 = Conv1D(filters=24,kernel_size=8,activation='relu')(l_lstm1)
l_drop_8 = Dropout(0.5)(l_conv_8)

conv_1 = [l_conv_7, l_conv_6, l_drop_8]

l_lstm_c = Concatenate(axis=1)(conv_1)

Second Half: CNN > LSTM

In [40]:
l_conv_4f = Conv1D(filters=12,kernel_size=4,activation='relu')(embedded_sequences_frozen)
l_drop_4f = Dropout(0.5)(l_conv_4f)
l_conv_4t = Conv1D(filters=12,kernel_size=4,activation='relu')(embedded_sequences_train)
l_drop_4t = Dropout(0.5)(l_conv_4t)

l_conv_3f = Conv1D(filters=12,kernel_size=3,activation='relu')(embedded_sequences_frozen)
l_conv_3t = Conv1D(filters=12,kernel_size=3,activation='relu')(embedded_sequences_train)

l_conv_2f = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_frozen)
l_conv_2t = Conv1D(filters=12,kernel_size=2,activation='relu')(embedded_sequences_train)

conv_2 = [l_drop_4f, l_drop_4t,l_conv_3f, l_conv_3t, l_conv_2f, l_conv_2t]

l_merge_2 = Concatenate(axis=1)(conv_2)
l_c_lstm = Bidirectional(LSTM(12,return_sequences=True))(l_merge_2)

In [41]:
l_merge = Concatenate(axis=1)([l_lstm_c, l_c_lstm])
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.5)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(12, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

In [42]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [43]:
!rm -r logs

rm: logs: No such file or directory


In [44]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [45]:
model.summary()
model.save('BalanceNet.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 30, 200)      6101200     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 30, 200)      6101200     input_4[0][0]                    
__________________________________________________________________________________________________
bidirectional_19 (Bidirectional (None, 30, 12)       9936        embedding_8[0][0]                
__________________________________________________________________________________________________
bidirectio

In [46]:
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 178s 6ms/step - loss: 1.3757 - acc: 0.4301 - val_loss: 1.2685 - val_acc: 0.4897
Epoch 2/200
 1800/32000 [>.............................] - ETA: 2:46 - loss: 1.2401 - acc: 0.5117

KeyboardInterrupt: 

In [ ]:
import pandas as pd
model.save('BalanceNet.h5')
pd.DataFrame(model_log.history).to_csv("history-balance.csv")